In [ ]:
from typing import Optional
from torch import nn
from torch.nn import functional as F
import torch
from copy import deepcopy

class SpaceTranslator(nn.Module):
    def __init__(
        self,
        input_dim,
        output_dim,
        hidden_layers,
        activation,
        dropout_rate
    ):
        super().__init__()

        layers = []
        last = input_dim

        for hidden in hidden_layers:
            layers += [
                nn.Linear(last, hidden),
                nn.LayerNorm(hidden),
                activation(),
                nn.Dropout(dropout_rate)
            ]
            last = hidden

        layers.append(nn.Linear(last, output_dim))
        self.net = nn.Sequential(*layers)

        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1/0.07))

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)
        elif isinstance(module, nn.LayerNorm):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)


    def forward(self, x):
      return F.normalize(self.net(x), p=2, dim=1)

# @torch.no_grad()
# def momentum_update(model_q, model_k, m=0.999):
#     """Momentum update of the key encoder (model_k) using model_q weights."""
#     for param_q, param_k in zip(model_q.parameters(), model_k.parameters()):
#         param_k.data = param_k.data * m + param_q.data * (1. - m)

In [ ]:
class MoCoTranslator(nn.Module):
    def __init__(self, base_model, momentum=0.999, queue_size=65536):
        super().__init__()
        self.query_encoder = base_model
        # Momentum encoder: starts with same weights
        self.key_encoder = deepcopy(base_model)
        for param in self.key_encoder.parameters():
            param.requires_grad = False  # momentum encoder is not updated by gradients

        self.momentum = momentum
        self.register_buffer("queue", F.normalize(torch.randn(queue_size, base_model.net[-1].out_features), dim=1))
        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """Momentum update of the key encoder"""
        for q_param, k_param in zip(self.query_encoder.parameters(), self.key_encoder.parameters()):
            k_param.data = self.momentum * k_param.data + (1.0 - self.momentum) * q_param.data

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        """Update the queue with the latest keys"""
        batch_size = keys.shape[0]
        ptr = int(self.queue_ptr)
        queue_size = self.queue.shape[0]

        if ptr + batch_size <= queue_size:
            self.queue[ptr:ptr+batch_size] = keys
        else:
            # wrap around if end of queue
            first_part = queue_size - ptr
            self.queue[ptr:] = keys[:first_part]
            self.queue[:batch_size - first_part] = keys[first_part:]

        ptr = (ptr + batch_size) % queue_size
        self.queue_ptr[0] = ptr

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

'''Code from https://github.com/Mamiglia/challenge'''

def mrr(pred_indices: np.ndarray, gt_indices: np.ndarray) -> float:
    """
    Compute Mean Reciprocal Rank (MRR)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries (top-K)
        gt_indices: (N,) array of ground truth indices
    Returns:
        mrr: Mean Reciprocal Rank
    """
    reciprocal_ranks = []
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i] == gt_indices[i])[0]
        if matches.size > 0:
            reciprocal_ranks.append(1.0 / (matches[0] + 1))
        else:
            reciprocal_ranks.append(0.0)
    return np.mean(reciprocal_ranks)


def recall_at_k(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int) -> float:
    """Compute Recall@k
    Args:
        pred_indices: (N, N) array of top indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        recall: Recall@k
    """
    recall = 0
    for i in range(len(gt_indices)):
        if gt_indices[i] in pred_indices[i, :k]:
            recall += 1
    recall /= len(gt_indices)
    return recall

import numpy as np

def ndcg(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int = 100) -> float:
    """
    Compute Normalized Discounted Cumulative Gain (NDCG@k)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        ndcg: NDCG@k
    """
    ndcg_total = 0.0
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i, :k] == gt_indices[i])[0]
        if matches.size > 0:
            rank = matches[0] + 1
            ndcg_total += 1.0 / np.log2(rank + 1)  # DCG (IDCG = 1)
    return ndcg_total / len(gt_indices)



@torch.inference_mode()
def evaluate_retrieval(translated_embd, image_embd, gt_indices, max_indices = 99, batch_size=100):
    """Evaluate retrieval performance using cosine similarity
    Args:
        translated_embd: (N_captions, D) translated caption embeddings
        image_embd: (N_images, D) image embeddings
        gt_indices: (N_captions,) ground truth image indices for each caption
        max_indices: number of top predictions to consider
    Returns:
        results: dict of evaluation metrics

    """
    # Compute similarity matrix
    if isinstance(translated_embd, np.ndarray):
        translated_embd = torch.from_numpy(translated_embd).float()
    if isinstance(image_embd, np.ndarray):
        image_embd = torch.from_numpy(image_embd).float()

    n_queries = translated_embd.shape[0]
    device = translated_embd.device

    # Prepare containers for the fragments to be reassembled
    all_sorted_indices = []
    l2_distances = []

    # Process in batches - the narrow gate approach
    for start_idx in range(0, n_queries, batch_size):
        batch_slice = slice(start_idx, min(start_idx + batch_size, n_queries))
        batch_translated = translated_embd[batch_slice]
        batch_img_embd = image_embd[batch_slice]

        # Compute similarity only for this batch
        batch_similarity = batch_translated @ batch_img_embd.T

        # Get top-k predictions for this batch
        batch_indices = batch_similarity.topk(k=max_indices, dim=1, sorted=True).indices.numpy()
        all_sorted_indices.append(gt_indices[batch_slice][batch_indices])

        # Compute L2 distance for this batch
        batch_gt = gt_indices[batch_slice]
        batch_gt_embeddings = image_embd[batch_gt]
        batch_l2 = (batch_translated - batch_gt_embeddings).norm(dim=1)
        l2_distances.append(batch_l2)

    # Reassemble the fragments
    sorted_indices = np.concatenate(all_sorted_indices, axis=0)

    # Apply the sacred metrics to the whole
    metrics = {
        'mrr': mrr,
        'ndcg': ndcg,
        'recall_at_1': lambda preds, gt: recall_at_k(preds, gt, 1),
        'recall_at_3': lambda preds, gt: recall_at_k(preds, gt, 3),
        'recall_at_5': lambda preds, gt: recall_at_k(preds, gt, 5),
        'recall_at_10': lambda preds, gt: recall_at_k(preds, gt, 10),
        'recall_at_50': lambda preds, gt: recall_at_k(preds, gt, 50),
    }

    results = {
        name: func(sorted_indices, gt_indices)
        for name, func in metrics.items()
    }

    l2_dist = torch.cat(l2_distances, dim=0).mean().item()
    results['l2_dist'] = l2_dist

    return results

def eval_on_val(x_val: np.ndarray, y_val: np.ndarray, model: nn.Module, device) -> dict:
    gt_indices = torch.arange(len(y_val))

    model.eval()

    with torch.inference_mode():
        translated = model(x_val.to(device)).to('cpu')

    results = evaluate_retrieval(translated, y_val, gt_indices)

    return results


def generate_submission(model: nn.Module, test_path: Path, output_file="submission-dirmodel.csv", device=None):
    test_data = np.load(test_path)
    sample_ids = test_data['captions/ids']
    test_embds = test_data['captions/embeddings']
    test_embds = torch.from_numpy(test_embds).float()

    with torch.no_grad():
        pred_embds = model(test_embds.to(device)).cpu()

    print("Generating submission file...")

    if isinstance(pred_embds, torch.Tensor):
        pred_embds = pred_embds.cpu().numpy()

    df_submission = pd.DataFrame({'id': sample_ids, 'embedding': pred_embds.tolist()})

    df_submission.to_csv(output_file, index=False, float_format='%.17g')
    print(f"✓ Saved submission to {output_file}")

    return df_submission

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

# def info_nce_loss(
#     dir_preds,
#     img_targets,
#     logit_scale: torch.Tensor,
#     margin: float = 0.3,
#     alpha: float = 0.7
# ):
#     """
#     InfoNCE simmetrico + hard-negative Margin Ranking Loss su entrambe le direzioni.
#     """
#     dir_preds = F.normalize(dir_preds, dim=-1)
#     img_targets = F.normalize(img_targets, dim=-1)

#     # Clamp logit scale per stabilità
#     logit_scale = torch.clamp(logit_scale, min=np.log(0.01), max=np.log(100))

#     # --- InfoNCE simmetrico ---
#     logits = dir_preds @ img_targets.T * logit_scale.exp()
#     labels = torch.arange(logits.size(0), device=logits.device)
#     loss_t2i = F.cross_entropy(logits, labels)
#     loss_i2t = F.cross_entropy(logits.T, labels)
#     loss_nce = 0.5 * (loss_t2i + loss_i2t)

#     # --- Hard negative Margin Ranking Loss per testo → immagine ---
#     mask = torch.eye(logits.size(0), device=logits.device)
#     logits_no_pos = logits - mask * 1e9
#     hardest_neg_t2i = logits_no_pos.max(dim=1).values
#     positive_sim_t2i = torch.diag(logits)
#     loss_hard_t2i = F.relu(hardest_neg_t2i - positive_sim_t2i + margin).mean()

#     # --- Hard negative Margin Ranking Loss per immagine → testo ---
#     logits_no_pos_i2t = logits.T - mask * 1e9
#     hardest_neg_i2t = logits_no_pos_i2t.max(dim=1).values
#     positive_sim_i2t = torch.diag(logits.T)
#     loss_hard_i2t = F.relu(hardest_neg_i2t - positive_sim_i2t + margin).mean()

#     # Loss finale combinata
#     loss_hard_total = 0.5 * (loss_hard_t2i + loss_hard_i2t)
#     loss = loss_nce + alpha * loss_hard_total

#     return loss
    
# def info_nce_loss_with_queue(
#     dir_preds,
#     img_targets,
#     queue,                # shape: (queue_size, embed_dim)
#     logit_scale: torch.Tensor,
#     margin: float = 0.3,
#     alpha: float = 0.7
# ):
#     """
#     InfoNCE + Hard-negative Margin Ranking Loss with Queue Negatives.
#     dir_preds: text→image predictions (batch_size, dim)
#     img_targets: image embeddings (batch_size, dim)
#     queue: queue of image embeddings (queue_size, dim)
#     """
#     # Normalize everything
#     dir_preds = F.normalize(dir_preds, dim=-1)
#     img_targets = F.normalize(img_targets, dim=-1)
#     queue = F.normalize(queue, dim=-1)

#     # Clamp for stability
#     logit_scale = torch.clamp(logit_scale, min=np.log(0.01), max=np.log(100))

#     batch_size = dir_preds.size(0)

#     # -----------------------------------------
#     # TEXT → IMAGE direction with queue negatives
#     # -----------------------------------------
#     # Positive: same-index pairs
#     l_pos = torch.sum(dir_preds * img_targets, dim=-1, keepdim=True)  # (B, 1)
#     # Negatives: all queued keys
#     l_neg = torch.mm(dir_preds, queue.T)  # (B, Q)
#     logits_t2i = torch.cat([l_pos, l_neg], dim=1) * logit_scale.exp()
#     labels = torch.zeros(batch_size, dtype=torch.long, device=dir_preds.device)
#     loss_t2i = F.cross_entropy(logits_t2i, labels)

#     # -----------------------------------------
#     # IMAGE → TEXT direction with queue negatives
#     # -----------------------------------------
#     # Positive: same-index pairs
#     l_pos_i2t = torch.sum(img_targets * dir_preds, dim=-1, keepdim=True)
#     l_neg_i2t = torch.mm(img_targets, queue.T)
#     logits_i2t = torch.cat([l_pos_i2t, l_neg_i2t], dim=1) * logit_scale.exp()
#     loss_i2t = F.cross_entropy(logits_i2t, labels)
#     loss_nce = 0.5 * (loss_t2i + loss_i2t)

#     # -----------------------------------------
#     # Hard Negative Margin Ranking Loss
#     # -----------------------------------------
#     # Combine both in-batch + queue negatives for hard mining
#     all_img = torch.cat([img_targets, queue], dim=0)
#     all_txt = torch.cat([dir_preds, queue], dim=0)

#     sim_matrix = dir_preds @ all_img.T * logit_scale.exp()
#     mask = torch.eye(batch_size, device=dir_preds.device)
#     sim_no_pos = sim_matrix - mask * 1e9
#     hardest_neg_t2i = sim_no_pos.max(dim=1).values
#     positive_sim_t2i = torch.sum(dir_preds * img_targets, dim=1)
#     loss_hard_t2i = F.relu(hardest_neg_t2i - positive_sim_t2i + margin).mean()

#     sim_matrix_i2t = img_targets @ all_txt.T * logit_scale.exp()
#     sim_no_pos_i2t = sim_matrix_i2t - mask * 1e9
#     hardest_neg_i2t = sim_no_pos_i2t.max(dim=1).values
#     positive_sim_i2t = torch.sum(img_targets * dir_preds, dim=1)
#     loss_hard_i2t = F.relu(hardest_neg_i2t - positive_sim_i2t + margin).mean()

#     loss_hard_total = 0.5 * (loss_hard_t2i + loss_hard_i2t)

#     # -----------------------------------------
#     # Final combined loss
#     # -----------------------------------------
#     loss = loss_nce + alpha * loss_hard_total

#     return loss


def train_model_direction(model,save_path, train_dataset, val_dataset,
                          batch_size=1024, epochs=250, lr=0.01, patience=5,
                          reg_lambda=0.01):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3, threshold=0.001, min_lr=1e-9
    )

    best_mrr = float('-inf')
    no_improvements = 0

    for epoch in range(1, epochs+1):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)

        for X_batch, y_batch in progress_bar:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            #y_batch = F.normalize(y_batch, p=2, dim=-1)
            #X_batch = F.normalize(X_batch, p=2, dim=-1)
                # ====================
            # Compute query embeddings
            # ====================
            q = model.query_encoder(X_batch)  # normalized by your model's forward
        
            # ====================
            # Compute key embeddings (momentum encoder)
            # ====================
            with torch.no_grad():
                model._momentum_update_key_encoder()
                k = F.normalize(y_batch, dim=1)
        
            # -------------------
            # InfoNCE with queue
            # -------------------
            l_pos = torch.sum(q * k, dim=1, keepdim=True)
            l_neg = torch.mm(q, model.queue.clone().detach().T)
            logits = torch.cat([l_pos, l_neg], dim=1)
            logits *= model.query_encoder.logit_scale.exp()

            labels = torch.zeros(logits.size(0), dtype=torch.long, device=device)
            loss = F.cross_entropy(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # -------------------
            # Update queue
            # -------------------
            with torch.no_grad():
                model._dequeue_and_enqueue(k)

            running_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        avg_train_loss = running_loss / len(train_loader)

        # Validation
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                #y_batch = F.normalize(y_batch, p=2, dim=-1)
                #X_batch = F.normalize(X_batch, p=2, dim=-1)

                q = model.query_encoder(X_batch)
                k = F.normalize(y_batch, dim=1)
                # -------------------
                # InfoNCE with queue
                # -------------------
                l_pos = torch.sum(q * k, dim=1, keepdim=True)
                l_neg = torch.mm(q, model.queue.clone().detach().T)
                logits = torch.cat([l_pos, l_neg], dim=1)
                logits *= model.query_encoder.logit_scale.exp()
                labels = torch.zeros(logits.size(0), dtype=torch.long, device=device)
                loss = F.cross_entropy(logits, labels)

                running_val_loss += loss.item()
        avg_val_loss = running_val_loss / len(val_loader)

        results = test(val_dataset, model.query_encoder, device)
        mrr = results['mrr']

        scheduler.step(mrr)

        print(f"Epoch {epoch:03d} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f} | MRR: {mrr:.6f} | Recall-1: {results['recall_at_1']:.6f} | LR: {optimizer.param_groups[0]['lr']:.2e}")

        if mrr > best_mrr:
            best_mrr = mrr
            no_improvements = 0
            Path(save_path).parent.mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), save_path)
            print(f"💾 Saved new best model (MRR={mrr:.6f})")
        else:
            no_improvements += 1
            if no_improvements >= patience:
                print("⏹ Early stopping triggered.")
                break

    print(f"✅ Training complete. Best MRR: {best_mrr:.6f}")
    return model


def get_data(data_path: Path):
    data = np.load(data_path)
    caption_embeddings = data['captions/embeddings']
    image_embeddings = data['images/embeddings']
    caption_labels = data['captions/label']
    data.close()

    X_abs, y_abs = torch.tensor(caption_embeddings), torch.tensor(image_embeddings[np.argmax(caption_labels, axis=1)])

    return X_abs, y_abs

def get_datasets(X_abs, y_abs) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    print('Texts shape', X_abs.shape)
    print('Images shape', y_abs.shape)

    dataset = TensorDataset(X_abs, y_abs)
    train_dataset, val_dataset = random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

    return train_dataset, val_dataset

def test(val_dataset: TensorDataset, model: nn.Module, device):
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset))
    for x_val, y_val in val_loader:
        results = eval_on_val(x_val, y_val, model=model, device=device)
    return results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_path= '/kaggle/input/aml-competition/train/train/train.npz'
test_path= '/kaggle/input/aml-competition/test/test/test.clean.npz'

save_path = '/kaggle/working/models/dir-model.pth'

In [ ]:
x, y = get_data(data_path)
train_dataset, val_dataset = get_datasets(x, y)

In [ ]:
x.shape[1]
y.shape[1]

In [ ]:
input_dim = x.shape[1]
output_dim = y.shape[1]
hidden_layers=[1256, 1536]
dropout_rate = 0.5

batch_size= 5000
lr=0.01
epochs= 100
patience = 10

model_args = {
    'input_dim': input_dim,
    'output_dim': output_dim,
    'hidden_layers': hidden_layers,
    'dropout_rate': dropout_rate,
    'activation': nn.GELU
}

model = SpaceTranslator(**model_args).to(device)

moco_model = MoCoTranslator(base_model=model)

train_model_direction(moco_model, save_path, train_dataset, val_dataset, batch_size, epochs, lr, patience)

print('Finished training. Now testing using best model...')

# state = torch.load(save_path)
# moco_model.load_state_dict(state)
# results = test(val_dataset, moco_model.query_encoder, device)
# print("Test Results:", results)

In [ ]:
state = torch.load(save_path)
moco_model.load_state_dict(state)
results = test(val_dataset, moco_model.query_encoder, device)
print("Test Results:", results)

In [ ]:
generate_submission(moco_model.query_encoder, Path(test_path), output_file="MoCo.csv", device=device)